In [1]:
# Crie um modelo determinístico para algum sistema de seu interesse. 
# Defina claramente o problema que o sistema representa e o que se deseja medir com uma simulação. 
# Faça um estudo sobre as condições de entrada e saída do modelo e execute algumas simulações do sistema. 
# Apresente os resultados da simulação de duas formas diferentes.

In [ ]:
import matplotlib.pyplot as plt